In [1]:
import pandas as pd
import numpy as np
import re

from io import StringIO

from sklearn.externals import joblib
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer


In [2]:
###читаем даатасет c фичами фильмов
df_f = pd.read_csv('./movie_titles.csv', names=['MovieID', 'Year', 'Title'])
df_f = df_f.set_index('MovieID')

###читаем даатасет с инфой о пользователях
f = open('./combined_data_1.txt')
file = f.read()

movieID_u = list(map(lambda x: int(x[:-1]), re.findall(r'\d+\:', file)))
df_u = re.split(r'\d+\:', file)
df_u = df_u[1:]    

df = []
for df_i, movieID_i in zip(df_u, movieID_u):
    sub_df = pd.read_csv(StringIO(df_i), names=['UserID', 'Score', 'Date'])
    sub_df['MovieID'] = movieID_i
    df.append(sub_df)


df = pd.concat(df)
df_f = df_f.drop(df_f[df_f.index>df.MovieID.unique().shape[0]].index)
#print(df_f)
#print(df)


In [3]:
u_size = df.UserID.unique().shape[0]
m_size = df.MovieID.unique().shape[0]
#y_size = df_f.Year.unique().shape[0]
#s_size = df.Date.unique().shape[0]

In [4]:
df.sort_values(by=['UserID', 'MovieID'], inplace=True)
df.reset_index(drop=True, inplace=True)
df_targ = df.Score

In [8]:
### разреженная матрица
sm = lil_matrix((df.shape[0], u_size + m_size))

In [9]:
j=0
temp = df.UserID.iat[0]

for i in range(df.shape[0]):
    
    if(temp<df.UserID.iat[i]):
        j+=1
        temp = df.UserID.iat[i]
        
    sm[i, j] = 1
    sm[i, u_size + df.MovieID.iat[i]-1] = 1
    
    if i % 50000 == 0:
        print("\r Progress: {}/{} ({}%)".format(i, df.shape[0], int(((i+1) / df.shape[0]) * 100)), end="")
    #print(i, j, sm[i,j])
    #print(i, j1, sm[i,j1])
#print(sm[0,0])

 Progress: 1600000/24053764 (6%)

KeyboardInterrupt: 

In [ ]:
sm = csr_matrix(sm)
joblib.dump(sm, 'sparse_df_1.bin')

In [5]:
sm = joblib.load('sparse_df_1.bin')

In [119]:
print(sm.shape)
print(df_targ.shape)

(24053764, 475257)
(24053764,)


По лемме 1 нелинейную часть можно представить в виде: $\sum_{i=1}^n \sum_{j=i+1}^n \langle {v}_i, {v}_f\rangle x_ix_j = \frac{1}{2}\sum_{f=1}^k \left( \left(\sum_{i=1}^n v_{i,f}x_i \right)^2 - \sum_{i=1}^n v^2_{i,f}x^2_i \right)$
<br>
Производная $\frac{dy}{dv}$ нелинейной части бдует следующей:
<br>
(1) = $\left( \left( \sum_{i=1}^n v_{i,f}x_i \right)^2 \right)' _{v_if} = 2\sum_{i=1}^n \left(v_{i,f}x_i \sum_{k=1}^n x_k\right)$ или в матричном виде $\sum_{f=1}^k2 (X\mathbb{1})^TXV_f$
<br>
(2) = $ \left( \sum_{i=1}^n v_{i,f}^2x_i^2 \right)' _{v_if} = 2\sum_{i=1}^n v_{i,f}x_i^2 $ или в матричном виде $\sum_{f=1}^k2 X^TXV_f$
<br>
<br>
Тогда
<br>
$\frac{dy}{dv_f} = \frac{1}{2}\sum_{f=1}^k \left( (1) - (2) \right)$ или в матричном виде $\sum_{f=1}^k(X\mathbb{1}-X)^TXV_f$
<br>
<br>
И тогда $\frac{dy}{dv}$ в матричном виде $(X\mathbb{1}-X)^TXV\mathbb{1}$

In [103]:
def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])


def narrTheta(lr, iter_num):
    return lr / np.sqrt(iter_num)


def sgdFFM(X, y, max_iter=1e3, step=1e-3, epsilon=1e-4, batch_size=256, k=9):
    w0 = 0
    iter_num = 1
    N = X.shape[0]
    np.random.seed(42)
    w1 = np.random.normal(size=X.shape[1])
    V = np.random.normal(size=(X.shape[1], k))

    while iter_num <= max_iter:
        random_batch = np.random.choice(N, batch_size)
        new_y = predict(X[random_batch, :], V, w0, w1)

        dy = 2 * (new_y - y[random_batch])
        print(dy.shape )
        w0 -= dy.mean() * step
        print(w0)
        
        if hasattr(X[random_batch, :], 'todense'):
            #print(np.multiply(dy, X[random_batch, :]))
            print(type(X[random_batch, :]), type(dy) )
            w1 -=  np.multiply(dy, X[random_batch, :]) * step
            #np.multiply(dy, X[random_batch, :]).mean(axis=0) * step
            print(w1)
        else:
            w1 -= X[random_batch, :].T @ dy * step / batch_size        
        
        for f in range(k):    
            if hasattr(X[random_batch, :], 'todense'):            
                dA = np.multiply(X[random_batch, :], (X[random_batch, :]@V[:,f]).reshape(-1, 1))            
                dB = (X[random_batch, :].power(2)).multiply(V[:,f])           
                V[:,f] -= step*np.asarray(((dA-dB).multiply(dy)).mean(axis=0))
            else:
                dA = X[random_batch, :]*(X[random_batch, :]@V[:,f]).reshape(-1, 1)          
                dB = (X[random_batch, :]**2)*V[:,f]        
                V[:,f] -= step*((dy.reshape(-1, 1)*(dA-dB)).mean(axis=0).T)
        
        iter_num += 1
        if (np.linalg.norm(y[random_batch] - new_y) < epsilon): 
            print(np.linalg.norm(y[random_batch] - new_y))
            return w0, w1, V
        
        if (iter_num%1000==0):
            print(np.linalg.norm(y[random_batch] - new_y))
        #print(np.linalg.norm(y[random_batch] - new_y))
        
    return w0, w1, V

def nonLin(X, V):
    
    if hasattr(X, 'todense'):  
        A = (X@V)**2
        B = (X.power(2))@V**2
    else:
        A = np.power(X@V,2)
        B = (np.power(X,2))@(V**2)
    
    return 1/2*((A-B)@np.ones(V.shape[1]))

def predict(X, V, w0, w1):
    return w0 + X @ w1 + nonLin(X, V)


In [ ]:
<h1>Проверим на генерации</h1>

In [55]:
X, y, coefs = make_regression(n_samples=10000, n_features=8, n_targets=1, n_informative=4, coef=True)

folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.concatenate((train, X[(i + 1) * fold_size:, :]))
            trainT = np.concatenate([trainT, y[(i + 1) * fold_size:]])
    
    Target = trainT
    Features = train
    w0, w1, V = sgdFFM(Features, Target, 1e4, 1e-3)

    train_pred = predict(Features, V, w0, w1)

    RMSE_train.append(RMSE(train_pred, Target))
    TargetT = testT
    FeaturesT = test
    test_pred = predict(FeaturesT, V, w0, w1)
    RMSE_test.append(RMSE(test_pred, TargetT))

print('Fold {}'.format(i + 1), '\nОбучающая: \n', train_pred - Target, '\nТестовая: \n', test_pred - TargetT, '\n', '\n')


194.10716989070156
29.067524427068978
3.836705860664591
0.5315016878376708
0.0730557964828968
0.009775829994820458
0.0013805060193305158
0.0001903245562204885
9.788449459047612e-05
185.032779795393
28.46664335491993
3.7911394730652637
0.5027443557658922
0.06698063126356317
0.010871223510680122
0.002141072896275881
0.00042757483761435126
9.980682242503832e-05
198.7386345420868
30.135044025068847
4.380551443363543
0.6352820718997864
0.07689296883146841
0.011588216208953863
0.001620178029509542
0.00024111056353326094
9.752735279247255e-05
198.6109693733482
28.623789421371182
4.32114405523216
0.5916651301456324
0.07142443237760764
0.011136358108735346
0.0016431975703725486
0.0002475356150605104
9.885363932779986e-05
187.39625093461277
27.231306169551306
3.7081497826934235
0.4778442866247861
0.0563844368084895
0.007713100762584396
0.0011648459522836908
0.00015770819335110794
9.74624683645014e-05
Fold 5 
Обучающая: 
 [ 1.61510259e-06 -9.84020048e-06  1.38342886e-05 ... -1.11486060e-05
  2.67

In [8]:
df = pd.DataFrame(np.vstack([RMSE_train, RMSE_test]), 
                  index=['rmse_train','rmse_test'])

df = pd.concat([df, df.mean(axis=1).rename('mean'),
                df.std(axis=1).rename('std')], axis=1)

df

,0,1,2,3,4,mean,std
rmse_train,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,2.394659e-07
rmse_test,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,1.846684e-07


<h1>Фигачим на данных1</h1>

In [100]:
print(sm.shape)
print(type(sm))
X = sm
y = df_targ.values

print(type(y))




(31, 5)
<class 'numpy.matrixlib.defmatrix.matrix'>
<class 'numpy.ndarray'>


In [ ]:
folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    print('{}-th fold'.format(i))
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.vstack([train, X[(i + 1) * fold_size:, :]])
            trainT = np.vstack([trainT, y[(i + 1) * fold_size:]])    
    
    
    Target = trainT
    Features = train
    w0, w1, V = sgdFFM(Features, Target, 1e3, 1e-3)

    train_pred = predict(Features, V, w0, w1)

    RMSE_train.append(RMSE(train_pred, Target))
    TargetT = testT
    FeaturesT = test
    test_pred = predict(FeaturesT, V, w0, w1)
    RMSE_test.append(RMSE(test_pred, TargetT))

print('Fold {}'.format(i + 1), '\nОбучающая: \n', train_pred - Target, '\nТестовая: \n', test_pred - TargetT, '\n', '\n')


In [ ]:
df = pd.DataFrame(np.vstack([RMSE_train, RMSE_test]), 
                  index=['rmse_train','rmse_test'])

df = pd.concat([df, df.mean(axis=1).rename('mean'),
                df.std(axis=1).rename('std')], axis=1)

df

<h1>Фигачим на генерации матриц</h1>

In [102]:
import random
M,N = 5,6

mat = np.matrix([1,0,0,0,0])
for i in range(5):
    matrix = [[random.randrange(0,7) for y in range(M)] for x in range(N)]
    mat = np.vstack([mat, matrix])
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if(mat[i, j]%2==0): 
            mat[i, j] = 0
        else:
            mat[i, j] = 1
Y = np.array([random.randrange(0,100) for y in range(mat.shape[0])])
print(type(Y))


<class 'numpy.ndarray'>


In [82]:
X = csr_matrix(mat)
y = Y.reshape(-1, 1)
folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    print('{}-th fold'.format(i))
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.vstack([train, X[(i + 1) * fold_size:, :]])
            trainT = np.vstack([trainT, y[(i + 1) * fold_size:]])
    print('Test fold {}: \n'.format(i),test.shape,'\n', test,'\n', testT.shape,'\n', testT,'\n','\n')
    print('Train fold {}: \n'.format(i),train.shape,'\n', train,'\n', trainT.shape,'\n', trainT,'\n','\n')

0-th fold
Test fold 0: 
 (6, 5) 
   (0, 0)	1
  (1, 1)	1
  (1, 3)	1
  (2, 1)	1
  (2, 3)	1
  (2, 4)	1
  (3, 1)	1
  (3, 2)	1
  (3, 4)	1
  (4, 0)	1
  (4, 1)	1
  (4, 4)	1
  (5, 2)	1
  (5, 4)	1 
 (6, 1) 
 [[89]
 [51]
 [62]
 [18]
 [82]
 [90]] 
 

Train fold 0: 
 (25, 5) 
   (0, 1)	1
  (0, 4)	1
  (1, 2)	1
  (1, 4)	1
  (2, 1)	1
  (3, 0)	1
  (4, 2)	1
  (4, 3)	1
  (5, 2)	1
  (5, 4)	1
  (6, 0)	1
  (6, 3)	1
  (6, 4)	1
  (7, 1)	1
  (7, 3)	1
  (7, 4)	1
  (8, 3)	1
  (9, 0)	1
  (10, 0)	1
  (10, 1)	1
  (10, 3)	1
  (11, 1)	1
  (11, 2)	1
  (11, 3)	1
  (11, 4)	1
  :	:
  (14, 4)	1
  (15, 0)	1
  (15, 1)	1
  (15, 3)	1
  (16, 1)	1
  (17, 4)	1
  (18, 0)	1
  (18, 1)	1
  (18, 3)	1
  (18, 4)	1
  (19, 0)	1
  (19, 1)	1
  (19, 2)	1
  (20, 0)	1
  (20, 1)	1
  (21, 0)	1
  (21, 1)	1
  (21, 2)	1
  (21, 4)	1
  (22, 2)	1
  (23, 1)	1
  (23, 2)	1
  (23, 3)	1
  (24, 1)	1
  (24, 2)	1 
 (25, 1) 
 [[29]
 [24]
 [19]
 [98]
 [36]
 [49]
 [84]
 [74]
 [10]
 [59]
 [ 4]
 [38]
 [29]
 [15]
 [35]
 [59]
 [ 5]
 [38]
 [60]
 [77]
 [65]
 [ 8]
 [

In [104]:

Target = trainT
Features = train
w0, w1, V = sgdFFM(Features, Target, 1e4, 1e-3, 1e-4, 8)

train_pred = predict(Features, V, w0, w1)

(8, 8)
0.0921709416169471
<class 'scipy.sparse.csr.csr_matrix'> <class 'numpy.matrixlib.defmatrix.matrix'>


TypeError: ufunc 'subtract' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''